In [14]:
import os
import uuid
import json
import requests
import fitz
import streamlit as st
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from typing import List, Dict
from openai import OpenAI

In [34]:
def read_pdf(file_path: str) -> str:
    with open(file_path, 'rb') as file:
        pdf_document = fitz.open(stream=file.read(), filetype="pdf")
        text = ""
        for page_num in range(pdf_document.page_count):
            page = pdf_document.load_page(page_num)
            text += page.get_text()
        return text

rpath = 'D:/pyprojects/New-file/rag_docs/Storey.pdf'
new_doc = read_pdf(rpath)

print(new_doc)

The Silent Locket
Chapter 1: The Symphony of Dust
The bell above the door of Thorne’s Curiosities didn’t just jingle; it sighed. It was a brass 
weary thing, forged in a Birmingham foundry in 1920, and it remembered every hand that 
had ever pushed it open. It remembered the rough callous of a long-dead delivery man, 
the gloved timidity of a widow selling her wedding china, and the sweaty palm of a thief 
who had once tried to lift a pocket watch in 1974.
Elias Thorne heard it all. Not the sound—the sound was merely a carrier wave—but the 
memory.
Elias was a listener. It was a condition without a medical name, a curse disguised as a 
talent. He didn’t listen to people—people lied, exaggerated, forgot, and rewrote their own 
histories to suit their egos. Elias listened to things. Matter, he had learned long ago, was 
the only reliable witness. Matter held onto resonance.
In his shop, the air was never truly quiet. To anyone else, Thorne’s Curiosities was a dusty, 
dimly lit emporium o

In [35]:
import pinecone
from pinecone import Pinecone
print(pinecone.__version__)  # Should be 5.x.x
print("Pinecone v5+ ready!")


8.0.0
Pinecone v5+ ready!


In [36]:
import pinecone
from pinecone import Pinecone  # New official client
import tiktoken
import openai
from chunking_evaluation.chunking import (
   RecursiveTokenChunker
)
from chunking_evaluation.utils import openai_token_count
import os


pc = Pinecone(api_key= 'pcsk_5C1T8A_QbkTs7cx31hXiL4W8WtoXji84kDK3qMgm4ZHhtKRXDGNrFrBwSHS7S19R7bwBiJ1')


In [44]:
import tiktoken  # make sure this is installed

def analyze_chunks(chunks, use_tokens=False):
    """
    Analyze overlap between two adjacent chunks.
    - By default looks at 200th and 201st chunks if they exist.
    - If there are fewer than 201 chunks, it uses the last two.
    """
    n = len(chunks)
    print("\nNumber of Chunks:", n)

    if n < 2:
        print("Not enough chunks to compare (need at least 2).")
        return

    # Pick indices: 199 & 200 if possible, otherwise last two
    if n > 200:
        idx1, idx2 = 199, 200
    else:
        idx1, idx2 = n - 2, n - 1
        print(f"Using last two chunks instead of 200th/201st (only {n} chunks).")

    chunk1, chunk2 = chunks[idx1], chunks[idx2]

    print("\n", "="*50, f"Chunk {idx1+1}", "="*50, "\n", chunk1)
    print("\n", "="*50, f"Chunk {idx2+1}", "="*50, "\n", chunk2)

    if use_tokens:
        encoding = tiktoken.get_encoding("cl100k_base")
        tokens1 = encoding.encode(chunk1)
        tokens2 = encoding.encode(chunk2)

        # Find overlapping tokens: suffix of chunk1 == prefix of chunk2
        max_len = min(len(tokens1), len(tokens2))
        for i in range(max_len, 0, -1):
            if tokens1[-i:] == tokens2[:i]:
                overlap = encoding.decode(tokens1[-i:])
                print("\n", "="*50, f"\nOverlapping text ({i} tokens):", overlap)
                return
        print("\nNo token overlap found")
    else:
        # Find overlapping characters: suffix of chunk1 == prefix of chunk2
        max_len = min(len(chunk1), len(chunk2))
        for i in range(max_len, 0, -1):
            if chunk1[-i:] == chunk2[:i]:
                print("\n", "="*50, f"\nOverlapping text ({i} chars):", chunk1[-i:])
                return
        print("\nNo character overlap found")


In [45]:
# remove  or comment out this manual test chunck cell

def chunk_text(document, chunk_size, overlap):
    chunks = []
    stride = chunk_size - overlap
    current_idx = 0
    
    while current_idx < len(document):
        # Take chunk_size characters starting from current_idx
        chunk = document[current_idx:current_idx + chunk_size]
        if not chunk:  # Break if we're out of text
            break
        chunks.append(chunk)
        current_idx += stride  # Move forward by stride
    
    return chunks

In [56]:
character_chunks = chunk_text(new_doc, chunk_size=52,overlap=0)
analyze_chunks(character_chunks)


Number of Chunks: 323

 ================================================== Chunk 200 ================================================== 
 .
"Hello?" he called out. His voice died instantly, 

 ================================================== Chunk 201 ================================================== 
 absorbed by the white mist.
The woman didn't turn. E

No character overlap found


In [47]:
import tiktoken

# Loading cl100k_base tokenizer 
encoder = tiktoken.get_encoding("cl100k_base")

# Text Example
text = "hamburger"
tokens = encoder.encode(text)

print("Tokens:", tokens)

Tokens: [71, 47775]


In [48]:
for i in range(len(tokens)):
    print(f"Token {i+1}:", encoder.decode([tokens[i]]))

print("Full Decoding: ", encoder.decode(tokens))

Token 1: h
Token 2: amburger
Full Decoding:  hamburger


In [50]:
def count_tokens(text, model="cl100k_base"):
    """Count tokens in a text string using tiktoken"""
    encoder = tiktoken.get_encoding(model)
    return print(f"Number of tokens: {len(encoder.encode(text))}")

In [58]:
fixed_token_chunker = FixedTokenChunker(
    chunk_size=52, 
    chunk_overlap=0,
    encoding_name="cl100k_base"
)

token_chunks = fixed_token_chunker.split_text(rpath)

analyze_chunks(token_chunks, use_tokens=True)


Number of Chunks: 1
Not enough chunks to compare (need at least 2).


In [61]:
recursive_character_chunker = RecursiveTokenChunker(
    chunk_size=200,  # Character Length
    chunk_overlap=0,  # Overlap
    length_function=len,  # Character length with len()
    separators=["\n\n", "\n", ".", "?", "!", " ", ""] # According to Research
)

recursive_character_chunks = recursive_character_chunker.split_text(new_doc)
analyze_chunks(recursive_character_chunks, use_tokens=False)


Number of Chunks: 103
Using last two chunks instead of 200th/201st (only 103 chunks).

 ================================================== Chunk 102 ================================================== 
 Elias closed his eyes and listened to the beautiful, terrible noise of the world. He would 
have to buy more lead. He would have to warn the others—if there were others like him.

 ================================================== Chunk 103 ================================================== 
 But for tonight, he just listened to the bayonet in the corner, faintly recovering its scream, 
and he thought it was the sweetest song he had ever heard.

No character overlap found


In [ ]:
from langchain_core.documents import Document
from datetime import datetime

recursive_character_chunker = RecursiveTokenChunker(
    chunk_size=200,
    chunk_overlap=0,
    length_function=len,
    separators=["\n\n", "\n", ".", "?", "!", " ", ""]
)

# STEP 1: split into raw text chunks
recursive_character_chunks = recursive_character_chunker.split_text(new_doc)

# STEP 2: convert into Document objects with metadata
docs = []
for i, chunk in enumerate(recursive_character_chunks):
    docs.append(
        Document(
            page_content=chunk,
            metadata={
                "chunk_id": i,
                "source": "myfile.pdf",
                "length": len(chunk),
                "timestamp": datetime.now().isoformat()  # add current time
            }
        )
    )

# OPTIONAL: analyze
ranalyze_chunks(recursive_character_chunks, use_tokens=False)



Number of Chunks: 103
Using last two chunks instead of 200th/201st (only 103 chunks).

 ================================================== Chunk 102 ================================================== 
 Elias closed his eyes and listened to the beautiful, terrible noise of the world. He would 
have to buy more lead. He would have to warn the others—if there were others like him.

 ================================================== Chunk 103 ================================================== 
 But for tonight, he just listened to the bayonet in the corner, faintly recovering its scream, 
and he thought it was the sweetest song he had ever heard.

No character overlap found
